In [ ]:
import mne
import sys

import matplotlib.pyplot as plt

sys.path.append('..')

from _parameters import *
sys.path.remove('..')

from IPython.display import clear_output

In [ ]:
def read_data(s):

    ses1 = dirs['eeg'] + '/eeg_s' + str(s) + 'a.bdf'
    ses2 = dirs['eeg'] + '/eeg_s' + str(s) + 'b.bdf'
        
    raw1 = mne.io.read_raw_bdf(ses1, preload=True)
    raw2 = mne.io.read_raw_bdf(ses2, preload=True)

    if s == 17:
        ses3 = dirs['eeg'] + '/eeg_s' + str(s) + 'c.bdf'
        raw3 = mne.io.read_raw_bdf(ses3, preload=True)
        raw = mne.concatenate_raws([raw1, raw2, raw3])
    else:
        raw = mne.concatenate_raws([raw1, raw2])

    return raw

In [ ]:
def mark_bad(raw, s):
    
    bad_chan = []

    # Marked during recording
    if bad_channels[s] != ['']:
        bad_chan += bad_channels[s]

    # Plot spectrum
    %matplotlib qt
    mne.viz.plot_raw_psd(raw, fmax = 40)

    # Wait for close fig
    while plt.get_fignums():
        plt.pause(1)

    # Input bad
    bad_psd = input('Channels to interpolate (_,_): ').split(sep=',')

    # Add if found
    if bad_psd != ['']:
        bad_psd = [str(c) for c in bad_psd]

        bad_chan += bad_psd
        bad_chan = list(set(bad_chan)) # Only unique channels

    # Mark as bad
    if bad_chan != []:
        raw.info['bads'] = bad_chan

    return raw    

In [ ]:
def preprocess(s: int):
    
    raw = read_data(s)

    # Re-reference
    raw.set_eeg_reference(channels['ref'])

    # Bipolar emg/eog
    raw = mne.set_bipolar_reference(raw, 'EXG3', 'EXG4', 'EOG',
                                    copy = False, drop_refs = True)
    raw = mne.set_bipolar_reference(raw, 'EXG5', 'EXG6', 'EMGL',
                                    copy = False, drop_refs = True)
    raw = mne.set_bipolar_reference(raw, 'EXG7', 'EXG8', 'EMGR',
                                    copy = False, drop_refs = True)

    raw.set_channel_types({'EOG':'eog'})
    raw.set_channel_types({'EMGL':'emg', 'EMGR':'emg'})

    # Drop
    raw.drop_channels(channels['drop'])

    # Montage
    biosemi64 = mne.channels.make_standard_montage('biosemi64')
    eeg_map = dict(zip(raw.info.ch_names[:64], biosemi64.ch_names))

    raw.rename_channels(eeg_map)
    raw.set_montage(biosemi64)

    # High pass filter EMG
    raw.filter(l_freq = 60, h_freq = None, picks = ['EMGL','EMGR'])

    # Mark bad channels
    raw = mark_bad(raw, s)
    print('Channels marked as bad:'); print(raw.info['bads'])

    # Interpolate
    raw.interpolate_bads()

    # Resample
    raw.resample(200)

    # Save clean raw
    raw_fname = dirs['raw'] + '/raw_s' + str(s) + '.fif'
    
    raw.save(raw_fname, overwrite = True)

    return raw

In [ ]:
# Run all subjects

for s in subjects:
    print('Running subject ' + str(s))
    raw = preprocess(s)
    clear_output(wait=False)

In [ ]:
# Run single subject

s = 17

print('Running subject ' + str(s))
raw = preprocess(s)